# LangChain RAG Example

In [ ]:
!pip install -q langchain sentence_transformers chromadb

In [ ]:
!pip install -q optimum auto-gptq

In [ ]:
!git clone https://github.com/cdfoundation/sig-mlops.git

fatal: destination path 'sig-mlops' already exists and is not an empty directory.


In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from itertools import chain

loader = DirectoryLoader('./sig-mlops', glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()

# MD-level splits
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_splits = list(chain.from_iterable([md_splitter.split_text(document.page_content) for document in documents]))

# Char-level splits
chunk_size = 250
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
splits = text_splitter.split_documents(md_splits)
print(f"# of loaded texts: {len(splits)}")

# of loaded texts: 1001


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_store = Chroma.from_documents(splits, embeddings)

In [ ]:
query = "What is the current trend of MLOps?"
docs = vector_store.similarity_search_with_score(query)
docs[-1]

(Document(page_content='This document sets out the current state of MLOps and provides a five year roadmap for future customer needs which is intended to support pre-competitive collaboration across the industry with a view to improving the overall state of MLOps as a', metadata={'Header 1': 'MLOps Roadmap 2022', 'Header 2': 'About this document'}),
 0.6282749772071838)

In [ ]:
!nvidia-smi

Wed Nov  1 15:23:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    42W /  70W |   1673MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline

model_id = "HuggingFaceH4/zephyr-7b-beta"
# model_id = "TheBloke/zephyr-7B-beta-GPTQ"
pipe = pipeline("text-generation", model=model_id, device_map="auto", **{"temperature": 0.5, "do_sample": True, "max_length": 1024, "max_new_tokens": 64})

# llm = HuggingFacePipeline.from_model_id(
#     model_id=model_id,
#     task="text-generation",
#     model_kwargs={"temperature": 0.5, "do_sample": True, "max_length": 64},
#     pipeline_kwargs={"device": 0},
# )


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())

query = "What is the current trend of MLOps?"
qa.run(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Both `max_new_tokens` (=64) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
